# 📘 Gap Down Recovery Strategy - NIFTY 500 Stocks
This notebook implements a trading strategy on NIFTY 500 stocks based on the idea of gap down recovery. The idea is to detect stocks that open significantly lower than the previous day (gap down), and enter a trade if they recover a certain percentage within a few days. We then monitor the trade to see if it hits a target profit or a stop loss.

### 🧩 Step 1: Import Required Libraries

pandas: For handling data tables (DataFrames).

numpy: For numerical operations.

yfinance: To download historical stock price data.

tqdm: To display progress bars during long loops.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm

### ⚙️ Step 2: Set Strategy Parameters

We define the rules of our trading strategy:

Look for a stock that gaps down more than 2%.

Enter the trade if it recovers at least 2%.

Exit when either:

Profit reaches 10%

Loss reaches 5%

Or after 60 days, whichever comes first.

In [2]:
gap_down_pct = -2.0     # Gap down: more than -2% from previous close
recovery_pct = 2.0      # Recovery: at least +2% from gap day close
tp_pct = 10.0           # Target Profit: +10%
sl_pct = -5.0           # Stop Loss: -5%
max_hold_days = 60      # Maximum number of days to hold a trade

### 📄 Step 3: Load NIFTY 500 Stock List

We load a CSV file containing a list of NIFTY 500 stock symbols that we want to analyze.

In [3]:
nifty_500 = pd.read_csv(r'C:\Users\Priyatanshu Ghosh\Documents\DS Practice\NIFTY500_2.csv')

### 🧮 Step 4: Prepare an Empty List for Trades

In [4]:
all_trades = []

This list will hold details of every trade taken by the strategy (entry, exit, result, etc.).

### 🔁 Step 5: Analyze Each Stock

We loop through each stock and:

Download 5 years of daily data using yfinance

Calculate if and when the stock gapped down and recovered

Simulate a trade based on our strategy

In [8]:
for stock in tqdm(nifty_500['Symbol']):
    
    try:
        # Download 5 years daily data
        df = yf.download(stock, period="5y", interval="1d", auto_adjust=True, progress=False)

        if df.shape[0] < 500:
            continue  # Skip if not enough data

        # Fix column names (IMPORTANT FIX)
        df.columns = df.columns.get_level_values(0)

        df.reset_index(inplace=True)
        
        # Add previous close column
        df['Prev_Close'] = df['Close'].shift(1)
        df = df.dropna()

# We have flattened the column names,
# Added a column for the previous day's closing price,
# Removed rows with missing values

        # Calculate Gap % (Open vs Prev Close)
        df.loc[:, 'Gap_Pct'] = ((df['Open'] - df['Prev_Close']) / df['Prev_Close']) * 100

# This tells us by how much the stock price gapped up or down compared to the previous day's close.


# 🔍 Detect Gap Down and Recovery
# We loop through each row in the DataFrame and:
# Look for a gap down of more than -2%
# Check if the stock recovers at least +2% after the gap
# Enter a trade on the recovery
# Monitor for up to 60 days and exit if:
# Profit ≥ 10% → WIN
# Loss ≤ -5% → LOSS
# Neither → Exit at 60 days → HOLD_EXIT     


        # Loop through rows
        for i in range(2, len(df)):
            
            # GAP DOWN detection
            if df.loc[i, 'Gap_Pct'] <= gap_down_pct:
                
                gap_day_close = df.loc[i, 'Close']
                
                # Look for recovery entry (same or later day)
                for j in range(i, len(df)):
                    
                    recovery_now_pct = ((df.loc[j, 'Close'] - gap_day_close) / gap_day_close) * 100
                    
                    if recovery_now_pct >= recovery_pct:
                        # ENTRY triggered
                        entry_date = df.loc[j, 'Date']
                        entry_price = df.loc[j, 'Close']
                        
                        trade_exit = False
                        
                        # Manage trade: next 60 days max
                        for k in range(j+1, min(j+1+max_hold_days, len(df))):
                            
                            pnl_pct = ((df.loc[k, 'Close'] - entry_price) / entry_price) * 100
                            
                            if pnl_pct >= tp_pct:
                                # Take profit hit
                                all_trades.append([
                                    stock, entry_date, entry_price,
                                    df.loc[k, 'Date'], df.loc[k, 'Close'],
                                    pnl_pct, (df.loc[k, 'Date'] - entry_date).days, 'WIN'
                                ])
                                trade_exit = True
                                break
                            
                            elif pnl_pct <= sl_pct:
                                # Stop loss hit
                                all_trades.append([
                                    stock, entry_date, entry_price,
                                    df.loc[k, 'Date'], df.loc[k, 'Close'],
                                    pnl_pct, (df.loc[k, 'Date'] - entry_date).days, 'LOSS'
                                ])
                                trade_exit = True
                                break
                        
                        # Force exit after max hold
                        if not trade_exit:
                            pnl_pct = ((df.loc[k-1, 'Close'] - entry_price) / entry_price) * 100
                            all_trades.append([
                                stock, entry_date, entry_price,
                                df.loc[k-1, 'Date'], df.loc[k-1, 'Close'],
                                pnl_pct, (df.loc[k-1, 'Date'] - entry_date).days, 'HOLD_EXIT'
                            ])
                        
                        break  # Only take first trade per gap

    except Exception as e:
        print(f"Error processing {stock}: {e}")

100%|██████████| 501/501 [05:15<00:00,  1.59it/s]


### 📝 Step 6: Convert All Trades to DataFrame

We store all trades in a structured DataFrame to analyze or export.

In [9]:
trades_df = pd.DataFrame(all_trades, columns=[
    'Stock', 'Entry_Date', 'Entry_Price',
    'Exit_Date', 'Exit_Price',
    'PnL_%', 'Days_in_Trade', 'Result'
])

### 📈 Step 7: Calculate Win Rate

We count:

Total trades taken

Number of winning trades

Win rate = (Wins ÷ Total Trades) × 100

In [10]:
total_trades = len(trades_df)
wins = trades_df[trades_df['Result'] == 'WIN'].shape[0]
win_rate = (wins / total_trades) * 100 if total_trades > 0 else 0

In [11]:
print(f"\nTotal Trades: {total_trades}")
print(f"Winning Trades: {wins}")
print(f"Win Rate: {win_rate:.2f}%")


Total Trades: 21131
Winning Trades: 9350
Win Rate: 44.25%


### 🗓️ Step 8: Format Dates for Excel and Export Results to Excel

In [13]:
trades_df['Entry_Date'] = pd.to_datetime(trades_df['Entry_Date']).dt.strftime('%Y-%m-%d')
trades_df['Exit_Date'] = pd.to_datetime(trades_df['Exit_Date']).dt.strftime('%Y-%m-%d')

This removes the time portion from dates, making them cleaner in the Excel file.

In [14]:
trades_df.to_excel("gap_down_recovery_trades.xlsx", index=False)

We save the final results to an Excel file named gap_down_recovery_trades.xlsx for future use.

In [15]:
print("\nExcel file saved: gap_down_recovery_trades.xlsx")


Excel file saved: gap_down_recovery_trades.xlsx


Confirmation that the Excel file has been saved